In [46]:
import chess.pgn
import chess.svg
import chess
import os
from flask import Flask, render_template, request
from natsort import natsorted
import shutil
import ast
import matplotlib.pyplot as plt
import os

# Connect to MongoDB
from pymongo import MongoClient
client = MongoClient('localhost', 27017) ## or MongoClient("localhost:27")
mongodb = client.chessDB
gameCollection = mongodb.gameCollection
playerCollection = mongodb.playerCollection

In [47]:
def getAllGames(name):
    query = { "$or": [
    {"headers.White": name},
    {"headers.Black": name}
    ]}
    documents = gameCollection.find(query)
    
    return documents

In [48]:
def getRatingPlot(player):
    x_values = player['dates'] 
    y_values = player['ratings'] 

    # Plot the graph
    plt.plot(x_values, y_values)
    plt.xlabel('Date')
    plt.ylabel('Rating')
    plt.title('Plot of rating vs date for {}'.format(player['name']))
    # save the plot to a file in the output directory
    filename = 'static/output/rating.png'
    plt.savefig(filename)

    # close the plot
    plt.close()

In [49]:
def getStats(name):
    query = {"name": name}
    documents = playerCollection.find(query)

    shutil.rmtree('static/output')

    # create the output directory if it doesn't already exist
    if not os.path.exists('static/output'):
        os.makedirs('static/output')

    for doc in documents:
        getRatingPlot(doc)
        return doc

In [50]:
def getGameplay(game):
    pgn = ''
    moves = ast.literal_eval(game)
    
    move_number = 1
    for i in range(1, len(moves), 2):
        pgn += str(move_number) + '. ' + moves[i-1] + ' ' + moves[i] + ' '
        move_number += 1

    pgn += '*'

    with open("static/game.pgn", "w") as file:
        file.write(pgn)

    folder_path = 'static/gameImages/'

    # Use shutil.rmtree() to delete everything from the folder
    shutil.rmtree(folder_path)

    # Recreate the folder if needed
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)


    pgn_file = open("static/game.pgn")
    pgn_game = chess.pgn.read_game(pgn_file)
    board = pgn_game.board()

    board_image = chess.svg.board(board=board)
    with open("static/gameImages/board0.svg", "w") as svg_file:
        svg_file.write(board_image)

    moveNumber = 1
    for move in pgn_game.mainline_moves():
        board.push(move)

        board_image = chess.svg.board(board=board)

        with open("static/gameImages/board{}.svg".format(moveNumber), "w") as svg_file:
            svg_file.write(board_image)
        moveNumber += 1


In [51]:
# game = '"d2d4", "g8f6", "c2c4", "g7g6", "g1f3", "f8g7", "g2g3", "c7c5", "d4d5", "e8g8", "f1g2", "d7d6", "b1c3", "b8a6", "e1g1", "a6c7", "a2a4", "a8b8", "e2e4", "a7a6", "a4a5", "b7b5", "a5b6", "b8b6", "f3d2", "e7e6", "f2f4", "e6d5", "c4d5", "f8e8", "d1c2", "c7b5", "d2c4", "b5d4", "c2d1", "b6b4", "c4d6", "d8d6", "e4e5", "d6b6", "e5f6", "g7f6", "c3e4", "f6g7", "c1d2", "b4b2", "f1e1", "e8d8", "d2a5", "b2g2", "g1g2", "b6b2", "d1d2", "b2d2", "e4d2", "d8f8", "a1c1", "c8b7", "c1c5", "d4e6", "c5c1", "b7d5", "g2f1", "h7h5", "d2c4", "h5h4", "c4e5", "h4h3", "a5b6", "f8e8", "c1d1", "d5b3", "d1d7", "b3a4", "d7b7", "a4b5", "f1f2", "g7e5", "e1e5", "e8c8", "g3g4", "e6f4", "f2g3", "f4e6", "g3h3", "c8c3", "b6e3", "e6f4", "h3g3", "f4d5", "b7b8", "g8g7", "e5d5", "c3e3", "g3f4", "e3e2", "g4g5", "b5e8", "d5d8", "g7f8", "b8a8", "e2e6", "d8b8", "e6e2", "a8a6", "e2h2", "a6a5", "h2e2", "a5e5", "e2e5", "f4e5", "f8e7", "b8b7", "e8d7", "b7a7"'
# getGameplay(game)

In [52]:
app = Flask("ChessGame", template_folder=r'templates')

@app.route('/')
def index():
    return render_template("Home.html")

@app.route('/gameSection', methods=['GET', 'POST'])
def getPlayerNames():
    options = []
    for document in playerCollection.find():
        options.append(document['name'])
    if request.method == 'POST':
        selectedPlayer = request.form.get('player_1_name')
        games = getAllGames(selectedPlayer)
        return render_template('ViewMatches.html', options=options, games=games, selectedPlayer=selectedPlayer)
    return render_template('ViewMatches.html', options=options)

@app.route('/<game>', methods=['GET'])
def getGameDetails(game):
    headers = request.args.get('headers')
    headers = ast.literal_eval(headers)
    getGameplay(game)
    image_names = []
    folder_path = "static/gameImages/"  # replace with the actual path to your folder
    files = os.listdir(folder_path)
    for file in files:
        if os.path.isfile(os.path.join(folder_path, file)):
            image_names.append('gameImages/'+file)
    image_names = natsorted(image_names)
    return render_template('GameReplay.html', image_names=image_names, headers=headers)


@app.route('/playerSection', methods=['GET', 'POST'])
def getPlayerStats():
    options = []
    for document in playerCollection.find():
        options.append(document['name'])
    if request.method == 'POST':
        selectedPlayer = request.form.get('player_1_name')
        stats = getStats(selectedPlayer)
        return render_template('ViewPlayers.html', options=options, stats=stats, selectedPlayer=selectedPlayer)
    return render_template('ViewPlayers.html', options=options)

@app.route('/contact')
def contact():
    return render_template('Contact.html')
    
app.run(host='localhost', port=5001)


 * Serving Flask app 'ChessGame' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [06/May/2023 17:24:00] "GET /playerSection HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2023 17:24:00] "GET /static/assets/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [06/May/2023 17:24:00] "GET /static/assets/js/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [06/May/2023 17:24:00] "GET /static/assets/js/jquery.dropotron.min.js HTTP/1.1" 304 -
127.0.0.1 - - [06/May/2023 17:24:00] "GET /static/assets/js/jquery.scrolly.min.js HTTP/1.1" 304 -
127.0.0.1 - - [06/May/2023 17:24:00] "GET /static/assets/js/jquery.scrollex.min.js HTTP/1.1" 304 -
127.0.0.1 - - [06/May/2023 17:24:00] "GET /static/images/tandon_long_color.png HTTP/1.1" 304 -
127.0.0.1 - - [06/May/2023 17:24:01] "GET /static/assets/css/fontawesome-all.min.css HTTP/1.1" 304 -
127.0.0.1 - - [06/May/2023 17:24:01] "GET /static/assets/js/browser.min.js HTTP/1.1" 304 -
127.0.0.1 - - [06/May/2023 17:24:01] "GET /static/assets/js/breakpoints.min.js HTTP/1.1" 304 -
127.0.0.1 - 